# YOLO Detection - Generalized Bounding Box Approach

In [7]:
!pip install -U mxnet-cu101==1.7.0

     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x663ec000 @  0x7f7575c7e615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 21kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [8]:
import numpy as np
from matplotlib import pyplot as plt

import cv2
import mxnet as mx

import pickle

import bounding_box as box

## Generalized Bounding Box Approach

In [1]:
vid_id = 'VIRAT_S_000201_08_001652_001838'

In [9]:
# unpickle file
pickle_in = open(vid_id + '-YOLO_results.pkl', 'rb')
classID_list, scores_list, boundingbox_list = pickle.load(pickle_in)

In [12]:
# count violations of social distancing given list of bounding boxes
count_violations = []
for idx, boxes in enumerate(boundingbox_list):
  if idx % 100 == 0:
    print("frame #:", idx)
  num_pairs, pair_ids, pairwise_distance_array = box.count_undistanced(boxes.asnumpy())
  count_violations.append(num_pairs)

frame #: 0
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
frame #: 100
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in this image!
No detections in th

In [16]:
print("Total # frames:", len(boundingbox_list))
print("# of occurrences of no social distancing:", sum(filter(None, count_violations)))

Total # frames: 5580
# of occurrences of no social distancing: 233


In [28]:
# replace None values with numpy nan
for idx, entry in enumerate(count_violations):
  if entry == None:
    count_violations[idx] = np.nan
count_violations = np.array(count_violations)

In [30]:
np.unique(count_violations[~np.isnan(count_violations)])

array([0., 1., 2., 3.])

In [38]:
index_dict = {}
for val in np.unique(count_violations[~np.isnan(count_violations)]):
  index_dict[val] = np.where(count_violations==val)[0]
index_dict[np.nan] = np.where(np.isnan(count_violations))[0]

In [39]:
index_dict

{0.0: array([   0,    1,    2, ..., 5577, 5578, 5579]),
 1.0: array([  13,   14,   15,   16,   17,   18,   20,   29,   32,   33,   34,
          35,   36,   49,   50,   51,  207,  209,  224,  228,  229,  261,
         262,  263,  264,  265,  272,  273,  278,  279,  280,  281,  282,
         283,  284,  285,  286,  287,  289,  290,  291,  292,  293,  294,
         305,  306,  309,  314,  315,  332,  333,  334,  338,  351,  377,
         378,  460,  463,  508,  509,  537,  550,  553,  569,  598,  600,
         608,  628,  629,  688,  814,  815,  816,  817,  818,  822,  846,
         847,  851,  852, 1478, 1481, 1497, 1935, 1956, 1958, 1959, 1962,
        1979, 1980, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1994,
        1995, 1996, 1998, 2003, 2095, 2097, 2204, 2205, 2206, 2207, 2208,
        2209, 2210, 2211, 2212, 2213, 2214, 2215, 2216, 2217, 2218, 2219,
        2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230,
        2284, 2502, 2503, 2505, 2510, 2551, 2555, 2